In [4]:
#Ovde je ideja da koristimo Neuralnu Mrezu koju smo istrenirali na prerecorded videima
#da je testiramo sa videa koji dolazi live sa webcamere

In [1]:
import os
import gdown
import cv2
import dlib
import imageio
import math
import time
import tensorflow as tf
import numpy as np

from matplotlib import pyplot as plt
from typing import List, Tuple
from tensorflow.python.client import device_lib

In [2]:
#varijable
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

global is_talking #upravo se desava govor
global last_talking_time #poslednji put kada je govor bio detektovan (u sekundama)
global flag_record #flag se aktivira ako su usta otvorena ili ako je proslo < 0.5 sekundi od zatvaranja usta(verovatno izgovaranje m ili p slova)
global prev_flag_record #da bi detektovali prelaz flag_record sa 1 na 0 i znamo da je gotova sekvenca pricanja i mozemo da obradimo podatke(funkcija procces_webcam)

flag_record = False 
is_talking = False
last_talking_time = 0
prev_flag_record = None

In [3]:
#Funkcija koja procesuira frejm kada je detektovan govor na web-kameri 
#i vraća tensor sa frejmovima govora, pripremljen za predikciju u neuronskoj mreži

def process_frame(detector, predictor, frame, width: int = 64, height: int = 64) -> List[float]: 
    
    global flag_record
    if flag_record:
        frames = []
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray_frame)
        lip_coords = None
    
        for face in faces:
            landmarks = predictor(gray_frame, face)
            
            lip_left = landmarks.part(48).x
            lip_right = landmarks.part(54).x
            lip_top = min(landmarks.part(50).y, landmarks.part(51).y)
            lip_bottom = max(landmarks.part(58).y, landmarks.part(59).y)
    
            lip_frame = frame[lip_top:lip_bottom, lip_left:lip_right]
            lip_frame_resized = cv2.resize(lip_frame, (width, height))  # Define `width` and `height`
            lip_frame_gray = cv2.cvtColor(lip_frame_resized, cv2.COLOR_BGR2GRAY)
    
            frames.append(lip_frame_gray)
            
            # Update lip_coords with actual coordinates if needed
            lip_coords = (lip_left, lip_right, lip_top, lip_bottom)
    
        if frames:
            frames = tf.convert_to_tensor(frames, dtype=tf.float32)
            mean = tf.reduce_mean(frames)
            std = tf.math.reduce_std(frames)
            frames = tf.cast((frames - mean), tf.float32) / std
            return frames, lip_coords
        else:
            return None, None
    else:
        return None, None
    
    

In [4]:
#Vizuelna reprezentacija na web-kameri koja pokazuje da je algoritam uspešno detektovao lice i izdvojio regiju usta

def process_frame_overlay(detector, predictor,  frame, width: int = 64, height: int = 64) -> np.ndarray: 
    
    global flag_record
    global is_talking, last_talking_time
    current_time = time.time()
        
    frames = []

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_frame)

    if len(faces) == 0:
        cv2.putText(frame, "No Face detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        
        for face in faces:
            
            landmarks = predictor(gray_frame, face)
            
            lip_left = landmarks.part(48).x
            lip_right = landmarks.part(54).x
            lip_top = min(landmarks.part(50).y, landmarks.part(51).y)
            lip_bottom = max(landmarks.part(58).y, landmarks.part(59).y)
    
            #print(f"Lip coordinates: left={lip_left}, right={lip_right}, top={lip_top}, bottom={lip_bottom}")
            mouth_top = (landmarks.part(51).x, landmarks.part(51).y)
            mouth_bottom = (landmarks.part(57).x, landmarks.part(57).y)
            lip_distance = math.hypot(mouth_bottom[0] - mouth_top[0], mouth_bottom[1] - mouth_top[1])
            
            lip_frame = frame[lip_top:lip_bottom, lip_left:lip_right]
            lip_frame_resized = cv2.resize(lip_frame, (width, height))
            lip_frame_gray = cv2.cvtColor(lip_frame_resized, cv2.COLOR_BGR2GRAY)
    
                 
            frames.append(lip_frame_gray)
            #print(lip_distance)
            if lip_distance >= 19.5:
                
                flag_record = True
                is_talking = True
               
                last_talking_time = current_time
                
                cv2.putText(frame, "Talking", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                for n in range(48, 61):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    cv2.circle(img=frame, center=(x, y), radius=3, color=(0, 0, 255), thickness=-1)
            
            else:
                
                if is_talking and (current_time - last_talking_time >= 0.5):

                    flag_record = False
                    is_talking = False
                    
                    cv2.putText(frame, "Silent", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    for n in range(48, 61):
                        x = landmarks.part(n).x
                        y = landmarks.part(n).y
                        cv2.circle(img=frame, center=(x, y), radius=3, color=(255, 0, 0), thickness=-1)
                else:
                    if is_talking:

                        flag_record = True
                        
                        cv2.putText(frame, "Talking", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        for n in range(48, 61):
                            x = landmarks.part(n).x
                            y = landmarks.part(n).y
                            cv2.circle(img=frame, center=(x, y), radius=3, color=(0, 0, 255), thickness=-1)
                    else:

                        flag_record = False
                        
                        cv2.putText(frame, "Silent", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        for n in range(48, 61):
                            x = landmarks.part(n).x
                            y = landmarks.part(n).y
                            cv2.circle(img=frame, center=(x, y), radius=3, color=(255, 0, 0), thickness=-1)
   
    return frame

In [5]:
#funkcija koja snima deo u kojem je detektovan govor i čuva ga kao GIF, 
#pružajući vizuelnu reprezentaciju podataka koji će biti korišćeni u neuronskoj mreži.

def save_talking_as_gif(collected_frames):
    
        frames_array = np.array(collected_frames)
        frames_array_uint8 = np.uint8(frames_array * 255)
        frames_array_uint8_squeezed = np.squeeze(frames_array_uint8)
        imageio.mimsave('./animation1.gif', frames_array_uint8_squeezed, fps=10)

In [6]:
#while loop

def process_webcam():
    
    global prev_flag_record
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return


    collected_frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        flipped_frame = cv2.flip(frame, 1)
        lip_frames, _ = process_frame(detector, predictor, flipped_frame) 
        processed_frame_overlay = process_frame_overlay(detector, predictor, flipped_frame) 
        
        if lip_frames is not None: 
            collected_frames.extend(lip_frames)
        
        if prev_flag_record == 1 and flag_record == 0:  #detektujemo prelaz sa 1 na 0 (tj prestanak govora i ponovo cutanje i znamo da mozemo da obradimo taj snimak)
                if collected_frames:
                    dataset = tf.data.Dataset.from_tensor_slices(collected_frames)
                    save_talking_as_gif(collected_frames) #vrati animation1.gif gde je snimljena sekvenca govora
                    print('ovde ide predikcija teksta...')               
                    collected_frames = [] #nakon predikcije teksta cistimo listu gde je bio govor za sledeci govor
                    
        prev_flag_record = flag_record 
        
        cv2.imshow('Webcam Overlay', processed_frame_overlay)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

   

In [7]:
process_webcam()  

ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
ovde ide predikcija teksta...
